In [ ]:
# import the relevant modules
import os
from netCDF4 import Dataset
import xarray as xr
import numpy as np

In [ ]:
# processing the observations
# trying to do it correctly

# path for the full .nc file
# will likely need to load this in chunks with dask
path_obs_nc = "/home/users/benhutch/ERA5_psl/ERA5-full-data.nc"

In [ ]:
# write a function to first select the iceland and azores //
# gridboxes
# also remap the obs data to a 2.5x2.5 degree grid
# first define the new grid
def create_new_grid():
    
    lon = np.arange(-180, 180, 2.5)
    lat = np.arange(-90, 90, 2.5)
    
    return xr.Dataset({'lon': (['lon'], lon),
                       'lat': (['lat'], lat)})

# define a function to select the data within a grid
def select_data_within_grid(dataset, grid):
    new_grid = create_new_grid()
    # interpolate dataset to new grid
    ds_interp = dataset.interp_like(new_grid, method='linear')  # Change 'linear' to 'nearest' for nearest neighbour interpolation
    return ds_interp.sel(lon=slice(grid['lon1'], grid['lon2']), lat=slice(grid['lat1'], grid['lat2']))

In [ ]:
# define the main function
def main(location, baseline):
    model_files = create_directory_structure(location)
    output_file = "/home/users/benhutch/ERA5_psl/{}-multi-model-ensemble-mean-state.nc".format(location)
    calculate_multi_model_ensemble_mean_state(model_files, output_file)

    grid = get_grid_spec(location)

    input_file = "/home/users/benhutch/ERA5_psl/adaptor.mars.internal-1669912474.882154-28712-3-70ecc8cf-50c6-4c45-967a-01d26ddaae07.nc"
    ds = xr.open_dataset(input_file, chunks={'time': 'auto'})  # Dask integrates with xarray via the chunks argument
    ds_within_grid = select_data_within_grid(ds, grid).compute()  # The computation isn't actually performed until you call .compute()

    # Write selected data to a new file
    output_file = "/home/users/benhutch/ERA5_psl/ERA5.{}-gridbox.psl.nc".format(location)
    ds_within_grid.to_netcdf(output_file)
